# Metody Obliczeniowe w Nauce i Technice
## Laboratorium 4 - Singular Value Decomposition (Wyszukiwarka)
### Albert Gierlach

### 1. Przygotowanie danych
Dane przygotowano za pomocą wiki-crawlera. Wykorzystano skrypt w Pythonie (https://github.com/bornabesic/wikipedia-crawler), dostosowując go do potrzeb zadania (dodanie opcji, która pozwala pobrać N artykułów). Źródła (wikipedia.py oraz crawler.py) są dostępne w archiwum z zadaniem.

Użycie:
```
python crawler.py N subdomain
```
gdzie N to liczba dokumentow do pobrania, a 'subdomain' to subdomena (użyto wartości 'en').
Dla polepszenia rezultatów zapewniono, że długość artykułu będzie większa niż 1000 znaków.

Dane w formacie .txt pobierane są do folderu ./data

### 2., 3. Określenie bag-of-words 
Stworzono klasę, która będzie przechowywać dane jednego dokumentu oraz metody pomagające w tym. Odrzucono kilka słów, które powinny zostać zignorowane podczas wyszukiwania artykułów. Całość operacji trwa około 3-5 minut. Program znajduje się w pliku lab4_preprocess.py

In [1]:
from collections import Counter
from scipy import sparse
import os
import pickle
import re
import operator
import numpy as np

data_dir = "./data"
cache_dir = "./cache"  # place for storing calculated matrices, etc

In [2]:
class ArticleData:
    ignored_words = ["a", "the", "of", "is"]  # and probably more

    def __init__(self, title):
        self.title = title.split('.')[0]
        self.bag_of_words = Counter()
        self.words_vec = None
        self.words_vec_norm = None

    def load_bag_of_words(self, path):
        with open(path, "rt", encoding='utf-8') as f:
            words = re.findall(r'\w+', f.read().lower())
            loaded_words = [word for word in words if len(word) > 2]
            self.bag_of_words.update(loaded_words)

        for ignore_token in ArticleData.ignored_words:
            del self.bag_of_words[ignore_token]

    def create_full_bag_of_words(self, keyset, size):
        self.words_vec = np.zeros(size)  # d_j
        for i, k in enumerate(keyset):
            self.words_vec[i] = self.bag_of_words[k]

        self.words_vec_norm = np.linalg.norm(self.words_vec)


In [3]:
articles_data = []
for file in os.listdir(data_dir):
    a_data = ArticleData(file)
    a_data.load_bag_of_words("{}/{}".format(data_dir, file))
    articles_data.append(a_data)

total_bag_of_words = Counter()
for article in articles_data:
    total_bag_of_words += article.bag_of_words

sizeof_total = len(total_bag_of_words)
wordset = list(total_bag_of_words.keys())
print("bag-of-words: {} words".format(sizeof_total))

total_bag_of_words_vector = np.zeros(sizeof_total, dtype=int)

# convert Counter to vectors
for counter, key in enumerate(wordset):
    total_bag_of_words_vector[counter] = total_bag_of_words[key]

for article in articles_data:
    article.create_full_bag_of_words(wordset, sizeof_total)
    
print("bag of words created!")

bag-of-words: 68581 words
bag of words created!


### 4., 5.  Rzadka macierz wektorów cech oraz IDF
Do budowy rzadkiej macierzy wykorzystano funckję crs_matrix(). Czas operacji 3-5 minut.

In [4]:
def getIDF(wordset, articles_data):
    articles_num = len(articles_data)
    idf = []
    for word in wordset:
        cnt = 0
        for article in articles_data:
            if article.bag_of_words[word] != 0:
                cnt += 1

        idf.append(np.log10(articles_num/cnt))

    return idf


def create_sparse(articles_data, sizeof_total, idf):
    row = []
    column = []
    data = []

    for i in range(len(articles_data)):
        article = articles_data[i]
        for j in range(sizeof_total):
            if article.words_vec[j] != 0:
                row.append(j)
                column.append(i)
                data.append(article.words_vec[j] * idf[j])


    term_by_document_matirx = sparse.csr_matrix((data, (row, column)), shape=(sizeof_total, len(articles_data)))
    return term_by_document_matirx

In [5]:
print('calculating idf')
idf = getIDF(wordset, articles_data)

print('creating sparse matrix')
term_by_document_matirx = create_sparse(articles_data, sizeof_total, idf)
print("term-by-document matrix: {}x{}".format(term_by_document_matirx.shape[0], term_by_document_matirx.shape[1]))

calculating idf
creating sparse matrix
term-by-document matrix: 68581x1500


**Tak wygenerowane wyniki (macierze, wektory) zapisano do plików, aby nie musieć każdorazowo obliczać dużych macierzy na nowo, jednak na potrzeby tego notebooka zapis został pominięty.
Wykorzystano bibliotekę dostarczaną z Pythonem - pickle. Rozmiar zapisanych plików wynosi około 800MB, gdyż głównie są to wektory, które posiadają dużo zer. **

### 6.  Program pozwalający na wyszukiwanie artykułów
Stworzono nowy plik lab4_search_engine.py, który będzie odpowiedzialny za przetwarzanie danych. Na początku wczytamy zcache'owane dane.

In [164]:
def do_query(query, k, word_list, articles):
    query = query.lower()
    words_dict = {word: index for index, word in enumerate(word_list)}
    words = re.findall(r'\w+', query)

    vec_query = np.zeros(len(word_list), dtype=int)
    for w in words:
        if w in words_dict.keys():
            vec_query[words_dict[w]] += 1

    if not np.any(vec_query):
        print("No results")
        return
    
    q_norm = np.linalg.norm(vec_query)
    res = []
    for a in articles:
        divider = q_norm * a.words_vec_norm
        prod = vec_query @ a.words_vec
        cos_theta = prod/divider
        res.append((cos_theta, a))

    res.sort(key=operator.itemgetter(0), reverse=True)
    for res_entry in res[:k]:
        print(res_entry[1].title)
        
        
        
# reassign variables, just for readibility
# articles - list with all of documents (words vectors + bag of words)
# word_list - bag_of_words_dict.keys()
# A - sparse matrix, columns are words vectors from articles_data
# bag_of_words - total bag_of_words vector (index 0 is occurences of the word word_list[0])
articles, word_list, A, bag_of_words = articles_data, wordset, term_by_document_matirx, total_bag_of_words_vector

In [165]:
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Text, Label, IntSlider, Output
from IPython.display import display, clear_output

def btn(b):
    output.clear_output()
    with output:
        how_many = form.children[0].children[1].value
        text_to_search = form.children[1].children[1].value
        if len(text_to_search) > 1:
            do_query(text_to_search, how_many, word_list, articles)
        else:
            output.append_stdout("Text is too short")

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

form_items = [
    Box([Label(value='Results num'), IntSlider(min=1, max=30, value=10, descritpion='k_')], layout=form_item_layout),
    Box([Label(value='Query'), Text(placeholder="Wpisz zapytanie", descritpion='query_')], layout=form_item_layout),
    Box([Label(), Button(description="Search!")], layout=form_item_layout)
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='50%'
))
output = Output()
form.children[2].children[1].on_click(btn)

In [166]:
form

Box(children=(Box(children=(Label(value='Results num'), IntSlider(value=10, max=30, min=1)), layout=Layout(dis…

In [168]:
display(output) # place for results

Output()